# bw2analyzer exploration functions

This notebook shows some new exploration functions addedd to `bw2analyzer`.

In [1]:
import bw2data as bd
import bw2analyzer as ba
import bw2calc as bc
import bw2io as bi

We use `ecoinvent` as an example database, but the functions shown here are generic.

In [2]:
bd.projects.set_current("ecoinvent 3.7.1 bw2")

## `print_recursive_supply_chain`

Sometimes it is convenient to print the supply chain of an activity. This function is only for exploration; use `bw2calc.GraphTraversal` in production.

In [3]:
ei = bd.Database("ecoinvent 3.7.1")

In [4]:
act = bd.get_activity(('ecoinvent 3.7.1', 'b28714960dbd0334840bbcecaf2c88c8'))
act

'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, PT, None)

In [5]:
ba.print_recursive_supply_chain(act)

1: 'heat and power co-generation, natural gas, conventional power plant, 
  0.246: 'market group for natural gas, high pressure' (cubic meter, Europe wit
    0.00324: 'market for natural gas, high pressure' (cubic meter, AT, None)
    0.0141: 'market for natural gas, high pressure' (cubic meter, BE, None)
    0.00525: 'market for natural gas, high pressure' (cubic meter, CZ, None)
    0.0496: 'market for natural gas, high pressure' (cubic meter, DE, None)
    0.000666: 'market for natural gas, high pressure' (cubic meter, DK, None)
    0.00749: 'market for natural gas, high pressure' (cubic meter, ES, None)
    0.00166: 'market for natural gas, high pressure' (cubic meter, FI, None)
    0.0145: 'market for natural gas, high pressure' (cubic meter, FR, None)
    0.036: 'market for natural gas, high pressure' (cubic meter, GB, None)
    0.0013: 'market for natural gas, high pressure' (cubic meter, GR, None)
    0.0028: 'market for natural gas, high pressure' (cubic meter, HU, None)
    0

This function also supports:

* Using a custom string instead of tabs for indentation
* Writing to a file-like object instead of printing to `stdout`
* Rescaling all amounts to an arbitrary value
* Using a cutoff to limit what is returned
* Specifying an arbitrary maximum recursion depth

Here is a silly example of most of these options:

In [6]:
ba.print_recursive_supply_chain(act, max_level=5, cutoff=0.02, tab_character="🐶", amount=2)

2: 'heat and power co-generation, natural gas, conventional power plant, 
🐶0.492: 'market group for natural gas, high pressure' (cubic meter, Europe wit
🐶🐶0.0282: 'market for natural gas, high pressure' (cubic meter, BE, None)
🐶🐶0.0993: 'market for natural gas, high pressure' (cubic meter, DE, None)
🐶🐶🐶0.0604: 'transport, pipeline, long distance, natural gas' (ton kilometer, DE, 
🐶🐶🐶0.0213: 'natural gas, high pressure, import from NL' (cubic meter, DE, None)
🐶🐶🐶0.0379: 'natural gas, high pressure, import from RU' (cubic meter, DE, None)
🐶🐶🐶🐶0.127: 'transport, pipeline, long distance, natural gas' (ton kilometer, RU, 
🐶🐶🐶🐶0.0332: 'transport, pipeline, long distance, natural gas' (ton kilometer, RER 
🐶🐶🐶🐶0.0379: 'natural gas production' (cubic meter, RU, None)
🐶🐶🐶🐶🐶0.0376: 'market for drying, natural gas' (cubic meter, GLO, None)
🐶🐶🐶0.0319: 'natural gas, high pressure, import from NO' (cubic meter, DE, None)
🐶🐶🐶🐶0.0319: 'petroleum and gas production, off-shore' (cubic meter, NO, None)
🐶🐶

## `print_recursive_calculation`

We can do the same thing, but filter not by the amounts consumed but their respective environmental impacts.

In [7]:
ipcc = ('IPCC 2013', 'climate change', 'GWP 100a')

In [8]:
ba.print_recursive_calculation(act, ipcc)

Fraction of score | Absolute score | Amount | Activity
0001 | 0.609 |     1 | 'heat and power co-generation, natural gas, conventional power plant, 
  0.148 | 0.09043 | 0.2459 | 'market group for natural gas, high pressure' (cubic meter, Europe wit
    0.0102 | 0.006202 | 0.01408 | 'market for natural gas, high pressure' (cubic meter, BE, None)
    0.0393 | 0.02394 | 0.04963 | 'market for natural gas, high pressure' (cubic meter, DE, None)
      0.0244 | 0.01485 | 0.01894 | 'natural gas, high pressure, import from RU' (cubic meter, DE, None)
    0.0241 | 0.0147 | 0.02472 | 'market for natural gas, high pressure' (cubic meter, IT, None)
      0.0176 | 0.01075 | 0.01325 | 'natural gas, high pressure, import from RU' (cubic meter, IT, None)
    0.0127 | 0.007755 | 0.05454 | 'market for natural gas, high pressure' (cubic meter, NO, None)


This function supports the same arguments as `print_recursive_supply_chain`.

## `find_differences_in_inputs`

Some databases have multiple activities that seem similar, but it is hard to tell how different they really are. `find_differences_in_inputs` is one of three functions that helps distinguish between different activities. It will look through the database that the activity came from, find all other activities with the same name and reference product, and see if their inputs are substantially different.

In our example activity, there are 60 different activities with the same name and reference product, so we limit the result to certain locations.

In [9]:
canada = ['CA-BC', 'CA-AB', 'CA-NS', 'CA-MB', 'CA-QC', 'CA-PE', 'CA-ON', 'CA-NB', 'CA-SK', 'CA-NT']

In [10]:
result = ba.find_differences_in_inputs(act, locations=canada)
result

{'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, PT, None): {'natural gas, high pressure': 0.245885675976663,
  'water, completely softened': 0.0575372481785392,
  'water, decarbonised': 1.91790827261797,
  'residue from cooling tower': -9.58954136308986e-06,
  'Hexane': 7.60450630093026e-06,
  'Sulfur dioxide': 5.4852176596874e-06,
  'Water': 0.05863365240771915,
  'Acetic acid': 1.16033450493388e-06,
  'Formaldehyde': 3.09742186027802e-07,
  'Butane': 8.87991530222121e-06,
  'Propane': 6.76062666097835e-06,
  'Carbon monoxide, fossil': 0.000191790827261797,
  'Carbon dioxide, fossil': 0.512081508788999,
  'Dinitrogen monoxide': 9.36898191173883e-06,
  'Water, cooling, unspecified natural origin': 0.0566582068869226,
  'Particulates, < 2.5 um': 4.70846480927712e-06,
  'PAH, polycyclic aromatic hydrocarbons': 7.67163309047189e-08,
  'Nitrogen oxides': 0.000351936168025398,
  'Propionic acid': 1.53432661809438e-07,
  'Pentane': 1.1

This function compares the net amount of each flow, and adds up multiple exchanges which reference the same flow. The above printed result is still a bit hard to interpret; returning this analysis result as a dataframe can allow for quick interpretation of the results:

In [29]:
ba.find_differences_in_inputs(act, locations=canada, as_dataframe=True)

,"natural gas, high pressure","water, completely softened","water, decarbonised",residue from cooling tower,Hexane,Sulfur dioxide,Water,Acetic acid,Formaldehyde,Butane,...,"Carbon dioxide, fossil",Dinitrogen monoxide,"Water, cooling, unspecified natural origin","Particulates, < 2.5 um","PAH, polycyclic aromatic hydrocarbons",Nitrogen oxides,Propionic acid,Pentane,Ethane,"Methane, fossil"
location,,,,,,,,,,,,,,,,,,,,,
PT,0.245886,0.057537,1.917908,-0.000010,0.000008,0.000005,0.058634,0.000001,3.097422e-07,0.000009,...,0.512082,0.000009,0.056658,0.000005,7.671633e-08,0.000352,1.534327e-07,0.000011,0.000013,0.000009
CA-NB,0.240821,0.056352,1.878406,-0.000009,0.000007,0.000005,0.057426,0.000001,3.033626e-07,0.000009,...,0.501534,0.000009,0.055491,0.000005,7.513625e-08,0.000345,1.502725e-07,0.000011,0.000013,0.000009
CA-SK,0.240821,0.056352,1.878406,-0.000009,0.000007,0.000005,0.057426,0.000001,3.033626e-07,0.000009,...,0.501534,0.000009,0.055491,0.000005,7.513625e-08,0.000345,1.502725e-07,0.000011,0.000013,0.000009
CA-MB,0.246926,0.057781,1.926020,-0.000010,0.000008,0.000006,0.058882,0.000001,3.110523e-07,0.000009,...,0.514247,0.000009,0.056898,0.000005,NaN,0.000353,NaN,0.000011,0.000013,0.000009
CA-NT,0.240821,0.056352,1.878406,-0.000009,0.000007,0.000005,0.057426,0.000001,3.033626e-07,0.000009,...,0.501534,0.000009,0.055491,0.000005,7.513625e-08,0.000345,1.502725e-07,0.000011,0.000013,0.000009
CA-AB,0.240821,0.056352,1.878406,-0.000009,0.000007,0.000005,0.057426,0.000001,3.033626e-07,0.000009,...,0.501534,0.000009,0.055491,0.000005,7.513625e-08,0.000345,1.502725e-07,0.000011,0.000013,0.000009
CA-PE,0.246926,0.057781,1.926020,-0.000010,0.000008,0.000006,0.058882,0.000001,3.110523e-07,0.000009,...,0.514247,0.000009,0.056898,0.000005,NaN,0.000353,NaN,0.000011,0.000013,0.000009
CA-ON,0.240821,0.056352,1.878406,-0.000009,0.000007,0.000005,0.057426,0.000001,3.033626e-07,0.000009,...,0.501534,0.000009,0.055491,0.000005,7.513625e-08,0.000345,1.502725e-07,0.000011,0.000013,0.000009
CA-QC,0.240821,0.056352,1.878406,-0.000009,0.000007,0.000005,0.057426,0.000001,3.033626e-07,0.000009,...,0.501534,0.000009,0.055491,0.000005,7.513625e-08,0.000345,1.502725e-07,0.000011,0.000013,0.000009


It is even easier to see clear patterns when the data is normalized:

In [14]:
df / df.iloc[0]

,"natural gas, high pressure","water, completely softened","water, decarbonised",residue from cooling tower,Hexane,Sulfur dioxide,Water,Acetic acid,Formaldehyde,Butane,...,"Carbon dioxide, fossil",Dinitrogen monoxide,"Water, cooling, unspecified natural origin","Particulates, < 2.5 um","PAH, polycyclic aromatic hydrocarbons",Nitrogen oxides,Propionic acid,Pentane,Ethane,"Methane, fossil"
location,,,,,,,,,,,,,,,,,,,,,
PT,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
CA-NB,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,...,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404
CA-MB,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,...,1.004230,1.004230,1.004230,1.004230,NaN,1.004230,NaN,1.004230,1.004230,1.004230
CA-NS,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,...,1.004230,1.004230,1.004230,1.004230,NaN,1.004230,NaN,1.004230,1.004230,1.004230
CA-AB,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,...,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404
CA-BC,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,...,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404
CA-ON,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,...,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404
CA-QC,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,...,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404
CA-PE,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,...,1.004230,1.004230,1.004230,1.004230,NaN,1.004230,NaN,1.004230,1.004230,1.004230


## `compare_activities_by_lcia_score`

A common question when trying to choose between different activities with similar sounding names is: Does it matter which one we choose? Are they actually any different? `compare_activities_by_lcia_score` allows for a comparison of any activities. If we look at very similar activities, we don't see a real difference:

In [31]:
justin = [
    a for a in ei 
    if a['name'] == act['name'] 
    and a['reference product'] == act['reference product']
    and a['location'] in canada
]
justin

['heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-SK, None),
 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-NT, None),
 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-PE, None),
 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-AB, None),
 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-QC, None),
 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-NB, None),
 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-MB, None),
 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-ON, None),
 'heat and power co-generation, natural gas, con

In [33]:
ba.compare_activities_by_lcia_score(
    [
        a for a in justin 
        if a['location'] != 'CA-QC'  # Je me souviens ;)
    ],
    ipcc
)

All activities similar


Allowing for Québec already produces different results:

In [34]:
ba.compare_activities_by_lcia_score(
    justin,
    ipcc
)

Differences observed. LCA scores:
	0.564 -> 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-SK, None)
	0.564 -> 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-NT, None)
	0.578 -> 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-PE, None)
	0.556 -> 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-AB, None)
	0.706 -> 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-QC, None)
	0.564 -> 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-NB, None)
	0.578 -> 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-MB, None)
	0.564 -> 'heat and power co-generation, natural gas, conventional power plan

Note that the Brightway developers take no position on Québec, Canada, Justin, or the correctness of the results provided above!

You can set the cutoff for what a "real" difference is with the `band` argument. Just for fun, let's look at many different CHP activities.

In [17]:
{act['name'] for act in ei if act['name'].startswith('heat and power co-generation, natural gas')}

{'heat and power co-generation, natural gas, 160kW electrical, Jakobsberg',
 'heat and power co-generation, natural gas, 160kW electrical, lambda=1',
 'heat and power co-generation, natural gas, 1MW electrical, lean burn',
 'heat and power co-generation, natural gas, 200kW electrical, lean burn',
 'heat and power co-generation, natural gas, 500kW electrical, lean burn',
 'heat and power co-generation, natural gas, 50kW electrical, lean burn',
 'heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical',
 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical',
 'heat and power co-generation, natural gas, mini-plant 2KW electrical'}

In [35]:
ba.compare_activities_by_lcia_score(
    [a for a in ei 
     if a['name'].startswith('heat and power co-generation, natural gas')
     and a['reference product'] == act['reference product']
    ], 
    ipcc,
    band=1
)

All activities similar


In [18]:
ba.compare_activities_by_lcia_score(
    [a for a in ei 
     if a['name'].startswith('heat and power co-generation, natural gas')
     and a['reference product'] == act['reference product']
    ], 
    ipcc,
    band=0.25
)

Differences observed. LCA scores:
	0.615 -> 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, GR, None)
	0.360 -> 'heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical' (kilowatt hour, IR, None)
	0.627 -> 'heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical' (kilowatt hour, HU, None)
	0.608 -> 'heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical' (kilowatt hour, SK, None)
	0.516 -> 'heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical' (kilowatt hour, LV, None)
	0.562 -> 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, SI, None)
	0.699 -> 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, TR, None)
	0.688 -> 'heat and power co-generation, natural gas, conventional power plant, 100MW elec

## `compare_activities_by_grouped_leaves`

Knowing that there are differences isn't always super helpful - one wants to *why* there are differences. We can't really go through the supply chains of many different activities manually, and keep all the differences straight. The function `compare_activities_by_grouped_leaves` will sum the *impacts* of the different supply chain activities by their [Common Product Classification](https://unstats.un.org/unsd/classifications/Econ/cpc) code. This will only work on databases which provide CPC codes in the same format as ecoinvent.

This function has multiple output formats. The default is a Python tuple of `(labels, data)`, but this is not shown here, as it is intended for programmatic use. Humans would choose either an HTML table:

In [21]:
ba.compare_activities_by_grouped_leaves(
    [a for a in ei 
     if a['name'].startswith('heat and power co-generation, natural gas')
     and a['reference product'] == act['reference product']
    ][:5], 
    ipcc,
    output_format="html",
)

Omitting activity name common prefix: 'heat and power co-generation, natural gas, '


activity,product,location,unit,total,direct emissions,"12020: Natural gas, liquefied or in the gaseous st",65131: Transport services via pipeline of petroleu,17100: Electrical energy,17300: Steam and hot water,53262: Power plants,65212: Coastal and transoceanic water transport se,39990: Other wastes n.e.c.,86211: Support services to oil and gas extraction,8621: Support services to mining,"4124: Bars and rods, hot-rolled, of iron or steel",53241: Long-distance pipelines,15310: Natural sands,54330: Excavating and earthmoving services,"34710: Polymers of ethylene, in primary forms","4128: Tubes, pipes and hollow profiles, of steel",53251: Local pipelines,531: Buildings,"374: Plaster, lime and cement",18000: Natural water,6424: Air transport services of passengers,"39270: Waste, parings and scrap of plastics"
"conventional power plant, 100MW electrical",,US-NPCC,kilowatt hour,0.590,0.818,0.097,0.013,0.036,0.021,0.007,0.000,0.002,0.002,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
"conventional power plant, 100MW electrical",,EE,kilowatt hour,0.563,0.847,0.093,0.047,0.000,0.001,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000
"combined cycle power plant, 400MW electrical",,RoW,kilowatt hour,0.554,0.837,0.093,0.020,0.011,0.000,0.004,0.006,0.000,0.004,0.000,0.003,0.001,0.001,0.001,0.001,0.001,0.000,0.001,0.000,0.000,0.000,0.000
"combined cycle power plant, 400MW electrical",,KR,kilowatt hour,0.492,0.880,0.083,0.000,0.012,0.000,0.003,0.002,0.006,0.006,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
"combined cycle power plant, 400MW electrical",,US-WECC,kilowatt hour,0.435,0.820,0.097,0.013,0.037,0.021,0.005,0.000,0.002,0.002,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


Or as a pandas `DataFrame`:

In [22]:
df = ba.compare_activities_by_grouped_leaves(
    [a for a in ei 
     if a['name'].startswith('heat and power co-generation, natural gas')
     and a['reference product'] == act['reference product']
    ][:5], 
    ipcc,
    output_format="pandas",
)
df

Omitting activity name common prefix: 'heat and power co-generation, natural gas, '


,activity,product,location,unit,total,direct emissions,17100: Electrical energy,65131: Transport services via pipeline of petroleum and natural gas,"12020: Natural gas, liquefied or in the gaseous state",17300: Steam and hot water,8621: Support services to mining,86211: Support services to oil and gas extraction,53262: Power plants,39990: Other wastes n.e.c.,53241: Long-distance pipelines,65212: Coastal and transoceanic water transport services of freight by tankers,53251: Local pipelines,18000: Natural water,39910: Municipal waste
0,"conventional power plant, 100MW electrical",,IT,kilowatt hour,0.686884,0.775616,0.081627,0.048086,0.045734,0.001161,0.010525,0.011150,0.004583,0.002157,0.003408,0.000407,0.000705,0.000242,0.000112
1,"combined cycle power plant, 400MW electrical",,HU,kilowatt hour,0.627076,0.689001,0.113822,0.096907,0.050409,0.001160,0.017464,0.013518,0.001168,0.002902,0.004554,0.000000,0.001518,0.000215,0.000186
2,"conventional power plant, 100MW electrical",,US-FRCC,kilowatt hour,0.590461,0.817801,0.036404,0.012908,0.096847,0.020536,0.000000,0.002113,0.007319,0.001556,0.002603,0.000000,0.000388,0.000259,0.000000
3,"conventional power plant, 100MW electrical",,CA-NB,kilowatt hour,0.563542,0.896115,0.019113,0.045732,0.024807,0.000000,0.000000,0.002101,0.007472,0.001457,0.001792,0.000000,0.000363,0.000146,0.000000
4,"conventional power plant, 100MW electrical",,TW,kilowatt hour,0.488965,0.877332,0.012010,0.000000,0.082746,0.000000,0.005302,0.005740,0.005966,0.006390,0.000000,0.002335,0.000332,0.000274,0.000000


What is included in this result? Direct emissions are those coming from the functional unit. Then, the function goes through the supply chain of each activity, and applies a cutoff criteria. If a given input matches the cutoff criteria (either it is too deep in the supply chain, or its impact is too small), we look up that inputs CPC code, and add it to any existing inputs with the same CPC code. We can also add impacts from direct emissions of an activity, even if it doesn't meet the cutoff criteria (if we didn't do this, the shares wouldn't sum to one).

As in the above functions, you can control how deep the search goes with the `max_level` and `cutoff` arguments. You can also switch from the default result type, which is fractional share of total impact, to absolute impact amounts, with the `mode` argument.

You can also use this function to explore the supply chain of a single activity. This is helpful when databases like ecoinvent provide many similar input activities (e.g. many electricity providers); it can be helpful to group by the product classification.

In [26]:
offset_printing = bd.get_activity(('ecoinvent 3.7.1', 'ff315bbf13be4ced226551a12cfdca53'))
offset_printing

'offset printing, per kg printed paper' (kilogram, CH, None)

In [27]:
ba.compare_activities_by_grouped_leaves(
    [offset_printing], 
    ipcc,
    output_format="html",
)

Omitting activity name common prefix: 'offset printing, per kg printed '


activity,product,location,unit,total,direct emissions,32113: Mechanical wood pulp; semi-chemical wood pu,41431: Unwrought aluminium,17100: Electrical energy,34790: Other plastics in primary forms; ion exchan,17300: Steam and hot water,6511: Road transport services of freight,32151: Corrugated paper and paperboard,34: Basic chemicals,"32112: Chemical wood pulp, other than dissolving g",15400: Clays,23220: Starches; inulin; wheat gluten; dextrins an,34240: Phosphates of triammonium; salts and peroxy,341: Basic organic chemicals,031: Wood in the rough,34231: Chemical elements n.e.c.; inorganic acids e,11040: Brown coal briquettes and similar solid fue,"12020: Natural gas, liquefied or in the gaseous st",35110: Paints and varnishes and related products,342: Basic inorganic chemicals n.e.c.,4153: Semi-finished products of aluminium or alumi,53269: Other constructions for manufacturing,6512: Railway transport services of freight,33370: Fuel oils n.e.c.,"34210: Hydrogen, nitrogen, oxygen, carbon dioxide",69120: Gas distribution through mains (on own acco,65229: Other inland water transport services of fr,161: Chemical and fertilizer minerals,"37420: Quicklime, slaked lime and hydraulic lime","34139: Other alcohols, phenols, phenol-alcohols, a",34280: Hydrogen peroxide; phosphides; carbides; hy,"34710: Polymers of ethylene, in primary forms",14290: Other non-ferrous metal ores and concentrat,11010: Hard coal,266: Woven fabrics (except special fabrics) of cot,"21611: Soya bean oil, crude",39283: Non-agglomerated wood waste and scrap,39950: Wastes from chemical or allied industries,33350: White spirit and special boiling point indu,39920: Sewage sludge,532: Civil engineering works,"21651: Palm oil, crude","34170: Ethers, alcohol peroxides, ether peroxides,",65119: Other road transport services of freight,34540: Oils and other products of the distillation,34800: Synthetic rubber and factice derived from o,"36330: Plates, sheets, film, foil and strip, of pl",53252: Local cables and related works,"444: Machinery for mining, quarrying and construct",39: Wastes or scraps,35130: Printing ink,"89200: Moulding, pressing, stamping, extruding and",39910: Municipal waste
paper,,CH,kilogram,2.092,0.000,0.227,0.162,0.135,0.058,0.043,0.040,0.031,0.029,0.028,0.024,0.021,0.019,0.018,0.017,0.016,0.015,0.014,0.011,0.009,0.008,0.007,0.006,0.005,0.005,0.004,0.003,0.003,0.003,0.002,0.002,0.002,0.002,0.002,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.011


In this particular instance, it was interesting for me to see the high fraction of impact coming from use of aluinium, which (probably) isn't the first thing one thinks of when considering printing.

In [39]:
df.set_index('location')

,"natural gas, high pressure","water, completely softened","water, decarbonised",residue from cooling tower,Hexane,Sulfur dioxide,Water,Acetic acid,Formaldehyde,Butane,...,"Carbon dioxide, fossil",Dinitrogen monoxide,"Water, cooling, unspecified natural origin","Particulates, < 2.5 um",Nitrogen oxides,Pentane,Ethane,"Methane, fossil","PAH, polycyclic aromatic hydrocarbons",Propionic acid
location,,,,,,,,,,,,,,,,,,,,,
PT,0.245886,0.057537,1.917908,-0.000010,0.000008,0.000005,0.058634,0.000001,3.097422e-07,0.000009,...,0.512082,0.000009,0.056658,0.000005,0.000352,0.000011,0.000013,0.000009,7.671633e-08,1.534327e-07
CA-MB,0.246926,0.057781,1.926020,-0.000010,0.000008,0.000006,0.058882,0.000001,3.110523e-07,0.000009,...,0.514247,0.000009,0.056898,0.000005,0.000353,0.000011,0.000013,0.000009,NaN,NaN
CA-AB,0.240821,0.056352,1.878406,-0.000009,0.000007,0.000005,0.057426,0.000001,3.033626e-07,0.000009,...,0.501534,0.000009,0.055491,0.000005,0.000345,0.000011,0.000013,0.000009,7.513625e-08,1.502725e-07
CA-QC,0.240821,0.056352,1.878406,-0.000009,0.000007,0.000005,0.057426,0.000001,3.033626e-07,0.000009,...,0.501534,0.000009,0.055491,0.000005,0.000345,0.000011,0.000013,0.000009,7.513625e-08,1.502725e-07
CA-NS,0.246926,0.057781,1.926020,-0.000010,0.000008,0.000006,0.058882,0.000001,3.110523e-07,0.000009,...,0.514247,0.000009,0.056898,0.000005,0.000353,0.000011,0.000013,0.000009,NaN,NaN
CA-NB,0.240821,0.056352,1.878406,-0.000009,0.000007,0.000005,0.057426,0.000001,3.033626e-07,0.000009,...,0.501534,0.000009,0.055491,0.000005,0.000345,0.000011,0.000013,0.000009,7.513625e-08,1.502725e-07
CA-BC,0.240821,0.056352,1.878406,-0.000009,0.000007,0.000005,0.057426,0.000001,3.033626e-07,0.000009,...,0.501534,0.000009,0.055491,0.000005,0.000345,0.000011,0.000013,0.000009,7.513625e-08,1.502725e-07
CA-SK,0.240821,0.056352,1.878406,-0.000009,0.000007,0.000005,0.057426,0.000001,3.033626e-07,0.000009,...,0.501534,0.000009,0.055491,0.000005,0.000345,0.000011,0.000013,0.000009,7.513625e-08,1.502725e-07
CA-NT,0.240821,0.056352,1.878406,-0.000009,0.000007,0.000005,0.057426,0.000001,3.033626e-07,0.000009,...,0.501534,0.000009,0.055491,0.000005,0.000345,0.000011,0.000013,0.000009,7.513625e-08,1.502725e-07


In [34]:
df.iloc[0]

location                                            PT
natural gas, high pressure                    0.245886
water, completely softened                    0.057537
water, decarbonised                           1.917908
residue from cooling tower                    -0.00001
Hexane                                        0.000008
Sulfur dioxide                                0.000005
Water                                         0.058634
Acetic acid                                   0.000001
Formaldehyde                                       0.0
Butane                                        0.000009
Propane                                       0.000007
Carbon monoxide, fossil                       0.000192
Carbon dioxide, fossil                        0.512082
Dinitrogen monoxide                           0.000009
Water, cooling, unspecified natural origin    0.056658
Particulates, < 2.5 um                        0.000005
Nitrogen oxides                               0.000352
Pentane   

In [20]:
act

'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, PT, None)

In [17]:
len(result)

60

Hmmm... that is frustrating, but unforunately common. Let's try "carboard" on its own.

In [6]:
ei.search("cardboard")

['treatment of waste plaster-cardboard sandwich, recycling' (kilogram, CH, None),
 'market for waste plaster-cardboard sandwich' (kilogram, GLO, None),
 'treatment of waste plaster-cardboard sandwich, sorting plant' (kilogram, CH, None),
 'treatment of waste plaster-cardboard sandwich, collection for final disposal' (kilogram, CH, None),
 'mandarin production, sorted and graded' (kilogram, ZA, None),
 'mandarin production, sorted and graded' (kilogram, ZA, None),
 'treatment of waste paperboard, unsorted, sorting' (kilogram, RoW, None),
 'tree seedling production, in unheated greenhouse' (unit, RER, None),
 'tree seedling production, in heated greenhouse' (unit, RER, None),
 'tree seedling production, in unheated greenhouse' (unit, RoW, None),
 'tree seedling production, in heated greenhouse' (unit, RoW, None),
 'treatment of waste paperboard, unsorted, sorting' (kilogram, CH, None),
 'treatment of waste paperboard, unsorted, sorting' (kilogram, Europe without Switzerland, None),
 'tre

Aapparently, the word "cardboard" is [deprecated](https://en.wikipedia.org/wiki/Paperboard), and we should say "paperboard" instead. What about "paperboard"?

In [7]:
ei.search("paperboard")

['market for waste paperboard' (kilogram, IN, None),
 'market for waste paperboard' (kilogram, ZA, None),
 'market for waste paperboard' (kilogram, PE, None),
 'market for waste paperboard' (kilogram, CO, None),
 'market for waste paperboard' (kilogram, BR, None),
 'market for waste paperboard' (kilogram, CY, None),
 'market for waste paperboard' (kilogram, MT, None),
 'market for waste paperboard' (kilogram, SE, None),
 'market for waste paperboard' (kilogram, NL, None),
 'market for waste paperboard' (kilogram, RS, None),
 'market for waste paperboard' (kilogram, FI, None),
 'market for waste paperboard' (kilogram, BA, None),
 'market for waste paperboard' (kilogram, DK, None),
 'market for waste paperboard' (kilogram, GB, None),
 'market for waste paperboard' (kilogram, MK, None),
 'market for waste paperboard' (kilogram, IS, None),
 'market for waste paperboard' (kilogram, LU, None),
 'market for waste paperboard' (kilogram, SI, None),
 'market for waste paperboard' (kilogram, IT, 

Not particularly helpful... maybe try from the other direction?

In [8]:
ei.search("box")

['packaging box factory construction' (unit, RoW, None),
 'packaging box factory construction' (unit, RER, None),
 'corrugated board box production' (kilogram, RER, None),
 'market for corrugated board box' (kilogram, CA-QC, None),
 'market for packaging box factory' (unit, GLO, None),
 'corrugated board box production' (kilogram, RoW, None),
 'market for corrugated board box' (kilogram, RoW, None),
 'corrugated board box production' (kilogram, CA-QC, None),
 'market for corrugated board box' (kilogram, RER, None),
 'carton board box production service, with gravure printing' (kilogram, CA-QC, None),
 'carton board box production service, with offset printing' (kilogram, CA-QC, None),
 'carton board box production service, with gravure printing' (kilogram, RoW, None),
 'carton board box production service, with gravure printing' (kilogram, CH, None),
 'market for carton board box production, with gravure printing' (kilogram, GLO, None),
 'carton board box production service, with offse

## A note on ecoinvent locations

The ecoinvent centre at some early point decided to invent its own location codes, and these are preserved for backwards compatibility, even though they are non-trivial for new users. You can read a technical manual on their [definition here](https://geography.ecoinvent.org/). For example, `RER` is the continent of Europe, and `RoW` is the "rest-of-the-world", which is defined as any place not covered by a country- or region-specific dataset for that particular activity. As this is a dynamic concept, there are [over 200 RoW possible definitions](https://geography.ecoinvent.org/rows/).

## A note on market versus transforming activities

TBD

## List comprehensions: An alternative to the `search` function

We can also use Python list comprehensions to do more powerful searches (more conditions, more data fields, etc.).

In [9]:
[act for act in ei if 'box' in act['name'] and act['location'] == 'RER']

['polycarboxylates production, 40% active substance' (kilogram, RER, None),
 'folding boxboard carton production' (kilogram, RER, None),
 'packaging box factory construction' (unit, RER, None),
 'market for folding boxboard carton' (kilogram, RER, None),
 'corrugated board box production' (kilogram, RER, None),
 'decarboxylative cyclization of adipic acid' (kilogram, RER, None),
 'folding boxboard carton production' (kilogram, RER, None),
 'corrugated board box production' (cubic meter, RER, None),
 'folding boxboard carton production' (kilogram, RER, None),
 'market for corrugated board box' (kilogram, RER, None),
 'carboxymethyl cellulose production, powder' (kilogram, RER, None),
 'decarboxylative cyclization of adipic acid' (kilogram, RER, None),
 'market for polycarboxylates, 40% active substance' (kilogram, RER, None)]

## General conclusions from searching

ecoinvent, and probably any similar technical database, can be tricky to use at first glance. The terms present are probably not what use in general speech, and finding what you want takes some time and creativity.

## Refining a list of possible activities

We aren't yet sure which of the following we want:

* 'corrugated board box production'
* 'carton board box production service, with gravure printing'
* 'carton board box production service, with offset printing'
* 'folding boxboard carton production'

Of course, we can look up the specific definitions, but we aren't sure how exactly they are implemented in ecoinvent. One trick is that we can see if they have different environmental impacts - if not, then it doesn't matter which one we choose.

In [10]:
NAMES = [
    'corrugated board box production',
    'carton board box production service, with gravure printing',
    'carton board box production service, with offset printing',
    'folding boxboard carton production',
]

boxes = [act for act in ei 
         if act['name'] in NAMES
         and act['unit'] == 'kilogram']

## `reference product` and multifunctional activities

TBD

In [11]:
[x['reference product'] for x in boxes]

['folding boxboard carton',
 'corrugated board box',
 'bark chips, wet, measured as dry mass',
 'carton board box production, with offset printing',
 'carton board box production, with gravure printing',
 'corrugated board box',
 'carton board box production, with gravure printing',
 'turpentine',
 'folding boxboard carton',
 'carton board box production, with offset printing',
 'turpentine',
 'bark chips, wet, measured as dry mass',
 'carton board box production, with offset printing',
 'carton board box production, with gravure printing',
 'corrugated board box']

In [12]:
boxes = [act for act in ei 
         if act['name'] in NAMES
         and act['unit'] == 'kilogram'
         and 'box' in act['reference product']]

In [13]:
[(x['name'], x['reference product'], x['location']) for x in boxes]

[('corrugated board box production', 'corrugated board box', 'RoW'),
 ('carton board box production service, with gravure printing',
  'carton board box production, with gravure printing',
  'CA-QC'),
 ('carton board box production service, with gravure printing',
  'carton board box production, with gravure printing',
  'RoW'),
 ('corrugated board box production', 'corrugated board box', 'RER'),
 ('folding boxboard carton production', 'folding boxboard carton', 'RER'),
 ('carton board box production service, with offset printing',
  'carton board box production, with offset printing',
  'RoW'),
 ('carton board box production service, with offset printing',
  'carton board box production, with offset printing',
  'CA-QC'),
 ('folding boxboard carton production', 'folding boxboard carton', 'RoW'),
 ('carton board box production service, with offset printing',
  'carton board box production, with offset printing',
  'CH'),
 ('corrugated board box production', 'corrugated board box', 'CA-

## ecoinvent documentation

TBD

In [14]:
[(x['name'], x['comment']) for x in boxes][:4]

[('corrugated board box production',
  'Geography:  This dataset is a weighted average built from the following geographies: US, RER, CA-QC.\nTechnology:  Average of present used technology\nTime period:  The production volume is valid for the year 2014. Inventory data are a weighted average stemming from different years between 2008 and 2014.'),
 ('carton board box production service, with gravure printing',
  'This dataset represents gravure printing of carton board box in Quebec. \nChild dataset exchanges identical to GLO dataset. Created to enable linking to regional markets (e.g. for electricity). Data quality indicators adjusted accordingly.\nThe input of cartonboard is not included into this module allowing the user to establish its own cartonboard boxes. Per kg of used cartonboard, 0.80 kg of this module are needed.\n[This dataset was already contained in the ecoinvent database version 2. It was not individually updated during the transfer to ecoinvent version 3. Life Cycle Imp

## Recommended Life Cycle Impact Assessment category

Make your life a bit easier, just use the latest IPCC factors with a 100 year time scale.

In this case study, as we are looking at biogenic carbon storage, we need to add non-fossil CO2 to the existing implementation.

In [109]:
ipcc_original = ('IPCC 2013', 'climate change', 'GWP 100a')
ipcc = ('IPCC 2013', 'climate change', 'GWP 100a', 'complete')

In [110]:
bio = bd.Database("biosphere3")
all_co2 = {x for x in bio if "carbon dioxide" in x['name'].lower()}

In [111]:
in_ipcc = {bd.get_activity(x) for x, _ in bd.Method(ipcc_original).load()}

In [112]:
all_co2.difference(in_ipcc)

{'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'low population density, long-term')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'lower stratosphere + upper troposphere')),
 'Carbon dioxide, non-fossil, from calcination' (kilogram, None, ['air']),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'urban air close to ground')),
 'Carbon dioxide, in air' (kilogram, None, ('natural resource', 'in air')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air',))}

In [113]:
gwp_complete_data = bd.Method(ipcc_original).load()

In [101]:
for flow in all_co2.difference(in_ipcc):
    if flow['categories'][0] == 'air':
        gwp_complete_data.append((flow.key, 1))
    else:
        gwp_complete_data.append((flow.key, -1))

In [107]:
gwp_complete = bd.Method(ipcc)
gwp_complete.register(
    description="IPCC 2013 100 year with biogenic carbon sources and sinks",
    unit='kg CO2-Eq',
)
gwp_complete.write(gwp_complete_data)

## Comparing activities by LCA score

In [16]:
ba.compare_activities_by_lcia_score(boxes, ipcc)

Differences observed. LCA scores:
	1.005'corrugated board box production' (kilogram, RoW, None)
	0.259'carton board box production service, with gravure printing' (kilogram, CA-QC, None)
	0.734'carton board box production service, with gravure printing' (kilogram, RoW, None)
	0.935'corrugated board box production' (kilogram, RER, None)
	1.778'folding boxboard carton production' (kilogram, RER, None)
	0.592'carton board box production service, with offset printing' (kilogram, RoW, None)
	0.372'carton board box production service, with offset printing' (kilogram, CA-QC, None)
	2.361'folding boxboard carton production' (kilogram, RoW, None)
	0.272'carton board box production service, with offset printing' (kilogram, CH, None)
	0.496'corrugated board box production' (kilogram, CA-QC, None)
	0.253'carton board box production service, with gravure printing' (kilogram, CH, None)


We have substantial differences for the *same* activity in *different* locations. We can now drill down and see where these differences come from. The following function will dive into the supply chain to see where differences arise, but this means it takes a bit longer than a simpler analysis.

In [17]:
ba.table_for_grouped_leaves_compared_activities(boxes, ipcc)

activity,product,location,unit,total,17100: Electrical energy,17300: Steam and hot water,32151: Corrugated paper and paperboard,35110: Paints and varnishes and related products,"32153: Cartons, boxes, cases, record sleeves and o",39910: Municipal waste,39240: Waste and scrap of paper or paperboard,35321: Soap; organic surface-active products and p,69120: Gas distribution through mains (on own acco,23220: Starches; inulin; wheat gluten; dextrins an,"12020: Natural gas, liquefied or in the gaseous st",32113: Mechanical wood pulp; semi-chemical wood pu,"32112: Chemical wood pulp, other than dissolving g","34710: Polymers of ethylene, in primary forms",341: Basic organic chemicals,39920: Sewage sludge,"33421: Ethylene, propylene, butylene, butadiene",6511: Road transport services of freight,53269: Other constructions for manufacturing,"21611: Soya bean oil, crude",333: Petroleum oils and oils obtained from bitumin,"37420: Quicklime, slaked lime and hydraulic lime",031: Wood in the rough,31230: Wood in chips or particles,34231: Chemical elements n.e.c.; inorganic acids e,"321: Pulp, paper and paperboard",65229: Other inland water transport services of fr,34790: Other plastics in primary forms; ion exchan,"01122: Maize (corn), other","34210: Hydrogen, nitrogen, oxygen, carbon dioxide",531: Buildings,35130: Printing ink,6512: Railway transport services of freight,"21651: Palm oil, crude",34240: Phosphates of triammonium; salts and peroxy,11010: Hard coal,41431: Unwrought aluminium,41122: Alloy steel in ingots or other primary form,33370: Fuel oils n.e.c.,34540: Oils and other products of the distillation,34: Basic chemicals,34280: Hydrogen peroxide; phosphides; carbides; hy,15400: Clays,11040: Brown coal briquettes and similar solid fue,3928: Sawdust and wood waste and scrap,"36330: Plates, sheets, film, foil and strip, of pl",543: Site preparation services,39283: Non-agglomerated wood waste and scrap,65119: Other road transport services of freight,"34270: Cyanides, cyanide oxides and complex cyanid","39270: Waste, parings and scrap of plastics",342: Basic inorganic chemicals n.e.c.,39990: Other wastes n.e.c.,39: Wastes or scraps,"353: Soap, cleaning preparations, perfumes and toi",53252: Local cables and related works,"34663: Herbicides, anti-sprouting products and pla","4121: Flat-rolled products of steel, not further w",347: Plastics in primary forms,161: Chemical and fertilizer minerals,"34139: Other alcohols, phenols, phenol-alcohols, a","482: Instruments and appliances for measuring, che","3466: Insecticides, fungicides, herbicides and dis",65213: Coastal and transoceanic water transport se,33380: Lubricants,53290: Other civil engineering works,"374: Plaster, lime and cement",34611: Urea,18000: Natural water,14290: Other non-ferrous metal ores and concentrat,4153: Semi-finished products of aluminium or alumi,33360: Gas oil,11050: Peat,53262: Power plants,65219: Other coastal and transoceanic water transp,"34720: Polymers of styrene, in primary forms",334: Petroleum gases and other gaseous hydrocarbon,"34651: Ammonia, anhydrous",266: Woven fabrics (except special fabrics) of cot,35499: Other chemical products n.e.c.,34250: Salts of oxometallic or peroxometallic acid,43420: Industrial or laboratory furnaces and ovens,39281: Wood pellets,431: Engines and turbines and parts thereof,391: Wastes from food and tobacco industry,"34520: Sulphur, except sublimed sulphur, precipita","461: Electric motors, generators and transformers,","34140: Carboxylic acids and their anhydrides, hali",53253: Sewage and water treatment plants,16390: Other minerals n.e.c.,34232: Phosphoric acid,"34170: Ethers, alcohol peroxides, ether peroxides,",34615: Mixtures of ammonium nitrate with calcium c,"42210: Reservoirs, tanks, vats and similar contain",34653: Ammonium chloride; nitrites,34220: Zinc oxide; zinc peroxide; chromium oxides,39950: Wastes from chemical or allied industries,64119: Other land transportation services of passe,34150: Amine-function com

Note that the values for each category as the *fraction* of total impact.

At this point, one would have to choose one of these product types. For the sake of this example we will go with "corrugated board box production". As this is in China, we choose the "RoW" location.

In [57]:
box = next(act for act in ei 
         if act['name'] == 'corrugated board box production'
         and act['location'] == 'RoW'
         and act['unit'] == 'kilogram')
box

'corrugated board box production' (kilogram, RoW, None)

## Examining a specific supply chain

TBD

In [58]:
ba.print_recursive_supply_chain(box, max_level=1)

1: 'corrugated board box production' (kilogram, RoW, None)
  0.465: 'market for containerboard, fluting medium' (kilogram, RoW, None)
  1.96e-06: 'market for butadiene' (kilogram, RoW, None)
  0.000459: 'market for acrylic varnish, without water, in 87.5% solution state' (
  3.36e-05: 'market for phenolic resin' (kilogram, RoW, None)
  8.88e-06: 'market for carbon dioxide, liquid' (kilogram, RoW, None)
  0.624: 'market for containerboard, linerboard' (kilogram, RoW, None)
  9.38e-05: 'market for propane, burned in building machine' (megajoule, GLO, None
  4.58e-05: 'market for perlite' (kilogram, GLO, None)
  0.00113: 'market for sodium hydroxide, without water, in 50% solution state' (k
  -6.21e-08: 'market for wastewater from maize starch production' (cubic meter, GLO
  -0.000538: 'market for sludge from pulp and paper production' (kilogram, RoW, Non
  1.96e-06: 'market for styrene' (kilogram, GLO, None)
  1.88e-05: 'market for vinyl acetate' (kilogram, GLO, None)
  -2.79e-05: 'marke

This is a bit much - lots of data, less actual useful information. Let's examine not just the raw inputs themselves, but the ones which contribute to the actual environmental impact.

In [59]:
ba.print_recursive_calculation(box, ipcc)

Fraction of score | Absolute score | Activity
1.000 (01.005): 'corrugated board box production' (kilogram, RoW, None)
  0.398 (0.4002): 'market for containerboard, fluting medium' (kilogram, RoW, None)
    0.334 (0.3357): 'containerboard production, fluting medium, recycled' (kilogram, RoW, 
      0.010 (0.01035): 'market for maize starch' (kilogram, GLO, None)
      0.029 (0.02876): 'market for waste paperboard, sorted' (kilogram, GLO, None)
      0.028 (0.02845): 'market for natural gas, low pressure' (cubic meter, RoW, None)
      0.024 (0.02383): 'market for sludge from pulp and paper production' (kilogram, RoW, Non
      0.037 (0.03693): 'market group for electricity, medium voltage' (kilowatt hour, RAS, No
      0.024 (0.02368): 'market for heat, district or industrial, other than natural gas' (meg
      0.016 (0.01604): 'market for municipal solid waste' (kilogram, RoW, None)
    0.044 (0.04383): 'containerboard production, fluting medium, semichemical' (kilogram, R
    0.013 (0

What conclusions can we draw this analysis?

* The supply chain for corrugated board production already has a significant fraction of recycled material from Asia (`RAS` in ecoinvent language).
* The recycled material appears to be used in the corrugated section (fluting medium), not the inside/outside liners. We could modify this if we have more specific information.
* We could consider modifying this supply chain to use the electricity and natural gas mixes present in China. But this probably requires some advanced techniques in the development branch of Brightway.
* We don't need to worry about any other inputs, they are small and we almost certainly won't get better data.

## Building an inventory database for our specific box

Best practice here is to create a new Brightway `Database` instance for each product under comparison, and use human-readable unique ID codes to make life easier. We don't define *how much* of each component here, but instead just the components themselves. We will need to have the following:

* The box itself
* Transport infrastructure: Pallet and container
* Truck transport in China and UK
* Printing
* Ocean transport
* Disposal: Ininceration or recycling

### Box itself

In [122]:
if "corrugated box" in bd.databases:
    del bd.databases["corrugated box"]

In [123]:
box_db = bd.Database("corrugated box")
box_db.register()  # Without this it doesn't exist in the metadata registry

In [124]:
def get_activity(**attributes):
    return next(act for act in ei if all(act[k] == v for k, v in attributes.items()))

In [125]:
def create_proxy_activity(input_activity, **attributes):
    act = box_db.new_activity(**attributes)
    act.save()
    
    exc = act.new_exchange(input=input_activity, amount=1, type="technosphere")
    exc.save()

In [126]:
create_proxy_activity(input_activity=box, code="box", location="GB", unit="kilogram", name="corrugated box")

### Pallet

In [127]:
ei.search("pallet")

['market for EUR-flat pallet' (unit, RER, None),
 'EUR-flat pallet production' (unit, RER, None),
 'market for EUR-flat pallet' (unit, RoW, None),
 'EUR-flat pallet production' (unit, RoW, None),
 'glass wool mat production, with phenolic binder, uncoated, Saint-Gobain ISOVER SA' (kilogram, CH, None),
 'glass wool mat production, with plant-based binder, uncoated, Saint-Gobain ISOVER SA' (kilogram, CH, None)]

In [128]:
ba.compare_activities_by_lcia_score([act for act in ei if act['name'] == 'market for EUR-flat pallet'], ipcc)

All activities similar


Use the global (`RoW`) one to be conservative.

In [129]:
pallet = get_activity(name = 'market for EUR-flat pallet', location= 'RoW')

In [130]:
create_proxy_activity(input_activity=pallet, code="pallet", location="GB", unit="unit", name="pallet")

### Container

In [131]:
ei.search("intermodal shipping container")

['operation, intermodal shipping container, 40-foot' (kg*day, GLO, None),
 'operation, intermodal shipping container, 20-foot' (kg*day, GLO, None),
 'intermodal shipping container production, 20-foot' (unit, GLO, None),
 'intermodal shipping container production, 40-foot' (unit, GLO, None),
 'operation, intermodal shipping container, 45-foot, high-cube' (kg*day, GLO, None),
 'operation, intermodal shipping container, 40-foot, high-cube' (kg*day, GLO, None),
 'maintenance, intermodal shipping container, 40-foot' (unit, GLO, None),
 'maintenance, intermodal shipping container, 20-foot' (unit, GLO, None),
 'treatment of used intermodal shipping container, 20-foot' (unit, GLO, None),
 'treatment of used intermodal shipping container, 40-foot' (unit, GLO, None),
 'market for operation, intermodal shipping container' (kg*day, GLO, None),
 'intermodal shipping container production, 40-foot, high-cube' (unit, GLO, None),
 'intermodal shipping container production, 45-foot, high-cube' (unit, GL

Check to make sure `operation` includes both production and maintenance

In [132]:
ba.print_recursive_supply_chain(get_activity(name = 'operation, intermodal shipping container, 20-foot'), max_level=1)

1: 'operation, intermodal shipping container, 20-foot' (kg*day, GLO, None
  1.04e-08: 'market for maintenance, intermodal shipping container, 20-foot' (unit
  1.04e-08: 'market for intermodal shipping container, 20-foot' (unit, GLO, None)


In [133]:
container = get_activity(name = 'operation, intermodal shipping container, 20-foot')

In [134]:
create_proxy_activity(input_activity=container, code="container", location="GB", unit="kilogram * day", name="container")

### Truck transport

In [135]:
ei.search("lorry")

['transport, freight, lorry, all sizes, EURO3 to generic market for transport, freight, lorry, unspecified' (ton kilometer, RER, None),
 'transport, freight, lorry, all sizes, EURO3 to generic market for transport, freight, lorry, unspecified' (ton kilometer, RoW, None),
 'transport, freight, lorry, all sizes, unregulated to generic market for transport, freight, lorry, unspecified' (ton kilometer, ZA, None),
 'transport, freight, lorry, all sizes, EURO1 to generic market for transport, freight, lorry, unspecified' (ton kilometer, ZA, None),
 'transport, freight, lorry, all sizes, EURO2 to generic market for transport, freight, lorry, unspecified' (ton kilometer, ZA, None),
 'market for transport, freight, lorry, unspecified' (ton kilometer, ZA, None),
 'transport, freight, lorry, all sizes, EURO4 to generic market for transport, freight, lorry, unspecified' (ton kilometer, RER, None),
 'transport, freight, lorry, all sizes, EURO4 to generic market for transport, freight, lorry, unspec

In the future, we should definintely revise this to reflect the lorry class and load factor. For now, we use the generic markets for `RoW` (travel in China) and `RER` (travel in UK).

In [136]:
create_proxy_activity(
    input_activity=get_activity(name='market for transport, freight, lorry, unspecified', location="RER"), 
    code="lorry GB", location="GB", unit="ton kilometer", name="lorry GB"
)
create_proxy_activity(
    input_activity=get_activity(name='market for transport, freight, lorry, unspecified', location="RoW"), 
    code="lorry CN", location="CN", unit="ton kilometer", name="lorry CN"
)

### Ocean freight

In [137]:
ei.search("ship ")

['container ship production' (unit, GLO, None),
 'maintenance, container ship' (unit, GLO, None),
 'market for container ship' (unit, GLO, None),
 'market for maintenance, container ship' (unit, GLO, None),
 'transport, freight, sea, container ship' (ton kilometer, GLO, None),
 'transport, freight, sea, container ship with reefer, cooling' (ton kilometer, GLO, None),
 'transport, freight, sea, container ship with reefer, freezing' (ton kilometer, GLO, None),
 'market for transport, freight, sea, container ship' (ton kilometer, GLO, None),
 'market for transport, freight, sea, container ship with reefer, freezing' (ton kilometer, GLO, None),
 'market for transport, freight, sea, container ship with reefer, cooling' (ton kilometer, GLO, None),
 'ferry production' (unit, GLO, None),
 'tanker production, for liquid goods other than petroleum and liquefied natural gas' (unit, GLO, None),
 'tanker production, for petroleum' (unit, GLO, None),
 'tanker production, for liquefied natural gas' (

In [138]:
create_proxy_activity(
    input_activity=get_activity(name='market for transport, freight, sea, container ship'), 
    code="ship", location="GLO", unit="ton kilometer", name="ship"
)

### Printing

In [139]:
ei.search("printing")

['carton board box production service, with gravure printing' (kilogram, CA-QC, None),
 'carton board box production service, with offset printing' (kilogram, CA-QC, None),
 'printing ink production, offset, product in 47.5% solution state' (kilogram, RER, None),
 'printing ink production, offset, product in 47.5% solution state' (kilogram, RoW, None),
 'printing ink production, rotogravure, product in 55% toluene solution state' (kilogram, RER, None),
 'printing ink production, rotogravure, product in 55% toluene solution state' (kilogram, RoW, None),
 'market for printing ink, offset, without solvent, in 47.5% solution state' (kilogram, RER, None),
 'market for printing ink, rotogravure, without solvent, in 55% toluene solution state' (kilogram, RER, None),
 'carton board box production service, with gravure printing' (kilogram, RoW, None),
 'carton board box production service, with gravure printing' (kilogram, CH, None),
 'market for carton board box production, with gravure printi

In [140]:
ba.print_recursive_supply_chain(get_activity(name='offset printing, per kg printed paper'), max_level=1)

1: 'offset printing, per kg printed paper' (kilogram, RoW, None)
  0.000398: 'market for thermoforming, with calendering' (kilogram, GLO, None)
  1.39: 'market for paper, woodfree, coated' (kilogram, RoW, None)
  0.000172: 'market for maize starch' (kilogram, GLO, None)
  0.0249: 'market for sheet rolling, aluminium' (kilogram, GLO, None)
  0.000179: 'market for synthetic rubber' (kilogram, GLO, None)
  0.00927: 'market for printing ink, offset, without solvent, in 47.5% solution s
  0.000218: 'market for industrial machine, heavy, unspecified' (kilogram, RoW, No
  0.00305: 'market for white spirit' (kilogram, GLO, None)
  0.000282: 'market for textile, woven cotton' (kilogram, GLO, None)
  0.0249: 'market for aluminium, wrought alloy' (kilogram, GLO, None)
  0.00119: 'market for acrylic dispersion, without water, in 65% solution state' 
  0.000391: 'market for acrylic varnish, without water, in 87.5% solution state' (
  0.00218: 'market for isopropanol' (kilogram, RoW, None)
  0.00052

This dataset is a combination of printing, and the material it is printed on. We need to separate the two.

In [141]:
import uuid
import copy

def copy_activity_and_exclude_some_inputs(original, exclusion_function=None, **attributes):
    if exclusion_function is None:
        exclusion_function = lambda x: False
        
    activity = bd.backends.peewee.proxies.Activity()
    for key, value in original.items():
        if key != "id":
            activity[key] = value
    for k, v in attributes.items():
        activity._data[k] = v
    if "code" not in activity._data:
        activity._data["code"] = str(code or uuid.uuid4().hex)
    activity.save()

    for exc in original.exchanges():
        if exclusion_function(exc):
            continue
        data = copy.deepcopy(exc._data)
        data["output"] = activity.key
        # Change `input` for production exchanges
        if exc["input"] == exc["output"]:
            data["input"] = activity.key
        bd.backends.peewee.schema.ExchangeDataset.create(**bd.backends.peewee.utils.dict_as_exchangedataset(data))
    return activity

In [142]:
copy_activity_and_exclude_some_inputs(
    original=get_activity(name='offset printing, per kg printed paper'), 
    exclusion_function=lambda x: any(string in x.input['name'] for string in ('paper', 'corrugated')), 
    database='corrugated box',
    code='printing',
    name='printing',
    location='CN',
)

'printing' (kilogram, CN, None)

### Incineration

In [143]:
ei.search("packaging incineration")

['treatment of waste packaging paper, municipal incineration' (megajoule, RoW, None),
 'treatment of waste packaging paper, municipal incineration' (kilowatt hour, RoW, None),
 'market for waste packaging paper' (kilogram, BR, None),
 'market for waste packaging paper' (kilogram, BE, None),
 'market for waste packaging paper' (kilogram, IS, None),
 'market for waste packaging paper' (kilogram, SK, None),
 'market for waste packaging paper' (kilogram, NO, None),
 'market for waste packaging paper' (kilogram, FI, None),
 'market for waste packaging paper' (kilogram, FR, None),
 'market for waste packaging paper' (kilogram, AT, None),
 'market for waste packaging paper' (kilogram, CH, None),
 'market for waste packaging paper' (kilogram, MT, None),
 'market for waste packaging paper' (kilogram, GR, None),
 'market for waste packaging paper' (kilogram, SE, None),
 'market for waste packaging paper' (kilogram, DK, None),
 'market for waste packaging paper' (kilogram, PT, None),
 'market for

Unfortunately, ecoinvent assumes that packaging paper is 100% recycled, and the incinceration datasets are actually for the production of heat (MJ) or electricity (kWh) from incineration.

In [144]:
ba.print_recursive_supply_chain(get_activity(name='market for waste packaging paper', location='GB'), max_level=1)

1: 'market for waste packaging paper' (kilogram, GB, None)
  0.0717: 'market for transport, freight, lorry, unspecified' (ton kilometer, RE
  1: 'waste packaging paper, Recycled Content cut-off' (kilogram, GLO, None


In [145]:
ei.search("paper* incineration")

['treatment of waste paperboard, municipal incineration' (kilogram, RoW, None),
 'treatment of waste paperboard, municipal incineration' (kilogram, CH, None),
 'market for waste paperboard' (kilogram, BR, None),
 'market for waste paperboard' (kilogram, MT, None),
 'market for waste paperboard' (kilogram, SE, None),
 'market for waste paperboard' (kilogram, NL, None),
 'market for waste paperboard' (kilogram, FI, None),
 'market for waste paperboard' (kilogram, DK, None),
 'market for waste paperboard' (kilogram, GB, None),
 'market for waste paperboard' (kilogram, IS, None),
 'market for waste paperboard' (kilogram, LU, None),
 'market for waste paperboard' (kilogram, SI, None),
 'market for waste paperboard' (kilogram, IT, None),
 'market for waste paperboard' (kilogram, DE, None),
 'market for waste paperboard' (kilogram, SK, None),
 'market for waste paperboard' (kilogram, PT, None),
 'market for waste paperboard' (kilogram, AT, None),
 'market for waste paperboard' (kilogram, BE, 

We will use `treatment of waste paperboard, municipal incineration`, even though this is a slightly different product, as we ddon't have a great alternative.

In [146]:
create_proxy_activity(
    input_activity=get_activity(
        name='treatment of waste paperboard, municipal incineration', 
        location='RoW',
        unit='kilogram'
    ), 
    code="incineration", location="GB", unit="kilogram", name="incineration"
)

### Recycling

In [147]:
create_proxy_activity(
    input_activity=get_activity(name='waste packaging paper, Recycled Content cut-off'), 
    code="recycling", location="GB", unit="kilogram", name="recycling"
)

## Building and interpreting a dynamic LCA

In [155]:
def functional_unit(mass, cn_truck_distance, uk_truck_distance, ocean_distance, ocean_time_in_days, treatment="recycling"):
    treatment_options = {
        'recycling': ("corrugated box", "recycling"),
        'incineration': ("corrugated box", "incineration"),
    }

    return {
        ("corrugated box", "box"): mass,
        ("corrugated box", "printing"): mass,
        treatment_options[treatment]: mass * (1 if treatment == 'recycling' else -1),
        ("corrugated box", "ship"): mass / 1000 * ocean_distance,
        ("corrugated box", "pallet"): (ocean_time_in_days + 30) / 365 / 5 / 1800, # Average lifetime is 5 years, 1800 kg weight limit
        ("corrugated box", "container"): mass * (ocean_time_in_days + 30),
        ("corrugated box", "lorry GB"): mass / 1000 * uk_truck_distance,
        ("corrugated box", "lorry CN"): mass / 1000 * cn_truck_distance,
    }

In [156]:
lca = bc.LCA(
    demand=functional_unit(
        mass=0.2, 
        cn_truck_distance=1.6 * 100, 
        uk_truck_distance=1.6 * 250, 
        ocean_distance=11076 * 1.852, # nautical miles to kilometers
        # 12 knots/hr; (nau mi * km / nau mi) / (12 knots/hr * km/knot) / hours/day
        ocean_time_in_days=(11076 * 1.852) / (12 * 1.852) / 24, # ~38
        treatment="recycling"
    ),
    method=ipcc
)
lca.lci()
lca.lcia()

In [157]:
lca.score

0.31369493632417994

In [158]:
recycling_config = functional_unit(
    mass=0.2, 
    cn_truck_distance=1.6 * 100, 
    uk_truck_distance=1.6 * 250, 
    ocean_distance=11076 * 1.852, # nautical miles to kilometers
    # 12 knots/hr; (nau mi * km / nau mi) / (12 knots/hr * km/knot) / hours/day
    ocean_time_in_days=(11076 * 1.852) / (12 * 1.852) / 24, # ~38
)

In [162]:
ba.print_recursive_calculation(bd.get_activity(('corrugated box', 'printing')), ipcc)

Fraction of score | Absolute score | Activity
1.000 (0.5283): 'printing' (kilogram, CN, None)
  0.049 (0.02612): 'market for printing ink, offset, without solvent, in 47.5% solution s
    0.049 (0.02589): 'printing ink production, offset, product in 47.5% solution state' (ki
      0.024 (0.0127): 'market for alkyd resin, long oil, without solvent, in 70% white spiri
  0.159 (0.08406): 'market for electricity, low voltage' (kilowatt hour, CH, None)
    0.150 (0.07912): 'electricity voltage transformation from medium to low voltage' (kilow
      0.150 (0.07912): 'market for electricity, medium voltage' (kilowatt hour, CH, None)
  0.024 (0.0129): 'sheet rolling, aluminium' (kilogram, RER, None)
    0.011 (0.00585): 'market group for electricity, medium voltage' (kilowatt hour, RER, No
      0.011 (0.005827): 'market group for electricity, medium voltage' (kilowatt hour, Europe 
  0.087 (0.04592): 'market for heat, district or industrial, natural gas' (megajoule, CH,
    0.030 (0.01578): '

In [162]:
ba.print_recursive_calculation(bd.get_activity(('corrugated box', 'printing')), ipcc)

Fraction of score | Absolute score | Activity
1.000 (0.5283): 'printing' (kilogram, CN, None)
  0.049 (0.02612): 'market for printing ink, offset, without solvent, in 47.5% solution s
    0.049 (0.02589): 'printing ink production, offset, product in 47.5% solution state' (ki
      0.024 (0.0127): 'market for alkyd resin, long oil, without solvent, in 70% white spiri
  0.159 (0.08406): 'market for electricity, low voltage' (kilowatt hour, CH, None)
    0.150 (0.07912): 'electricity voltage transformation from medium to low voltage' (kilow
      0.150 (0.07912): 'market for electricity, medium voltage' (kilowatt hour, CH, None)
  0.024 (0.0129): 'sheet rolling, aluminium' (kilogram, RER, None)
    0.011 (0.00585): 'market group for electricity, medium voltage' (kilowatt hour, RER, No
      0.011 (0.005827): 'market group for electricity, medium voltage' (kilowatt hour, Europe 
  0.087 (0.04592): 'market for heat, district or industrial, natural gas' (megajoule, CH,
    0.030 (0.01578): '

In [159]:
for k, v in recycling_config.items():
    lca.redo_lcia({k: v})
    print(k, lca.score)

('corrugated box', 'box') 0.1539705422272089
('corrugated box', 'printing') 0.1056659317408352
('corrugated box', 'recycling') 0.0
('corrugated box', 'ship') 0.038388738683975986
('corrugated box', 'pallet') -0.0008524594708889591
('corrugated box', 'container') 0.0016516624990677061
('corrugated box', 'lorry GB') 0.010500018552574555
('corrugated box', 'lorry CN') 0.004370502091406333


In [160]:
lca.redo_lcia(functional_unit(
    mass=0.2,
    cn_truck_distance=1.6 * 100, 
    uk_truck_distance=1.6 * 250, 
    ocean_distance=11076 * 1.852, # nautical miles to kilometers
    # 12 knots/hr; (nau mi * km / nau mi) / (12 knots/hr * km/knot) / hours/day
    ocean_time_in_days=(11076 * 1.852) / (12 * 1.852) / 24, # ~38
    treatment="incineration"
))

In [161]:
lca.score

0.6358969823904835

# Why use Brightway & custom software versus other packaging LCA solutions?

Our value proposition is *improved accuracy*. We can accomplish this with software designed to treat datasets as computation models instead of text documents:

* Make it easy to add or combine multiple data sources, both manually and programmatically
* Make it possible to modify the supply chain, both at the surface, and deep down, both persistently and in memory
* Create programmtic tools to examine the supply chain, to help to better identify which products to use when modelling specific supply chains.
* Allow for performant uncertainty and global sensitivity analysis (e.g. dimension reduction from 300k uncertain parameters to 50, providing decision support to customers on what drive variation in environmental performance).